# Emerging Technologies - Problems

This notebook contains solutions to the problems assigned for the Emerging Technologies module.

**Author:** Tommy Mitchell  
**Module:** Emerging Technologies  
**Institution:** ATU

## Problem 1: Generating Random Boolean Functions

The **Deutsch–Jozsa algorithm** is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either:

- **Constant**: Always returns `False` or always returns `True` for all inputs
- **Balanced**: Returns `True` for exactly half of the possible input combinations

Question 1 is to write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

---

### Background and Research

The Deutsch-Jozsa algorithm, developed by David Deutsch and Richard Jozsa in 1992[1], represents one of the first demonstrations of quantum computational advantage. The promblem is esoteric being more of a proof of concept, but it provides a clear example where a quantum algorithm exponentially outperforms any deterministic classical algorithm.

**Classical vs Quantum Complexity:**
- **Classical (deterministic)**: In the worst case, we need to query $2^{n-1} + 1$ inputs to determine if an n-bit function is constant or balanced
- **Quantum**: Only **1 query** is needed, regardless of the number of input bits

For a 4-bit input function worst case:
- Classical: 9 queries
- Quantum: 1 query

**References:**
1. Deutsch, D., & Jozsa, R. (1992). "Rapid solution of problems by quantum computation." *Proceedings of the Royal Society of London A*, 439(1907), 553-558.
2. Nielsen, M. A., & Chuang, I. L. (2010). *Quantum Computation and Quantum Information*. Cambridge University Press.
3. IBM Qiskit Textbook: [Deutsch-Jozsa Algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms)

### Understanding Constant and Balanced Functions

For a function $f: \{0,1\}^n \rightarrow \{0,1\}$ with 4 Boolean inputs ($n=4$), there are $2^4 = 16$ possible input combinations.

**Constant functions** (2 total):
- $f(x) = 0$ for all inputs (always False)
- $f(x) = 1$ for all inputs (always True)

**Balanced functions**: Return True for exactly 8 of the 16 inputs, and False for the other 8. The number of such functions is $\binom{16}{8} = 12,870$.

Our `random_constant_balanced` function will randomly select either a constant or balanced function and return it as a callable Python function.

### Implementation: `random_constant_balanced` Function

The implementation strategy:
1. Randomly choose to generate a constant or balanced function
2. If constant: randomly choose True or False as the return value
3. If balanced: randomly select exactly 8 of the 16 possible inputs to return True

We use closures to capture the function's behavior, allowing the returned function to remember which inputs map to True.

In [7]:
import random
from itertools import product
from typing import Callable


def random_constant_balanced() -> Callable[[bool, bool, bool, bool], bool]:
    """
    Generate a random function that is either constant or balanced.
    
    The function takes four Boolean arguments and returns a single Boolean.
    
    Constant functions:
        - Always return True, OR
        - Always return False
    
    Balanced functions:
        - Return True for exactly half (8) of the 16 possible inputs
        - Return False for the other half
    
    Returns:
        A callable function f(b1, b2, b3, b4) -> bool that is either
        constant or balanced.
    
    Example:
        >>> f = random_constant_balanced()
        >>> f(True, False, True, False)  # Returns True or False
    """
    # Generate all 16 possible input combinations for 4 Boolean arguments
    all_inputs = list(product([False, True], repeat=4))
    
    # Randomly decide: constant (True) or balanced (False)
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Constant function: always returns the same value
        constant_value = random.choice([True, False])
        
        def constant_function(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:
            """A constant function that always returns the same value."""
            return constant_value
        
        # Store metadata for testing/verification
        constant_function.function_type = "constant"
        constant_function.constant_value = constant_value
        return constant_function
    
    else:
        # Balanced function: returns True for exactly half the inputs
        # Randomly select 8 inputs that will return True
        true_inputs = set(random.sample(all_inputs, 8))
        
        def balanced_function(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:
            """A balanced function that returns True for exactly half the inputs."""
            return (b1, b2, b3, b4) in true_inputs
        
        # Store metadata for testing/verification
        balanced_function.function_type = "balanced"
        balanced_function.true_inputs = true_inputs
        return balanced_function

### Demonstration and Verification

We can test our `random_constant_balanced` function by:
1. Generating several random functions
2. Evaluating them on all 16 possible inputs
3. Verifying if they are constant or balanced

In [8]:
def verify_function(f: Callable[[bool, bool, bool, bool], bool]) -> str:
    """
    Verify whether a function is constant or balanced by evaluating all inputs.
    
    Args:
        f: A function taking four Boolean arguments
        
    Returns:
        A string describing the function type and its outputs
    """
    all_inputs = list(product([False, True], repeat=4))
    outputs = [f(*inp) for inp in all_inputs]
    
    true_count = sum(outputs)
    false_count = len(outputs) - true_count
    
    if true_count == 0:
        return f"CONSTANT (always False) - True: {true_count}, False: {false_count}"
    elif true_count == 16:
        return f"CONSTANT (always True) - True: {true_count}, False: {false_count}"
    elif true_count == 8:
        return f"BALANCED - True: {true_count}, False: {false_count}"
    else:
        return f"INVALID - True: {true_count}, False: {false_count}"


# Generate and test 10 random functions
print("Testing random_constant_balanced() function:\n")
print("-" * 60)

for i in range(10):
    f = random_constant_balanced()
    result = verify_function(f)
    func_type = getattr(f, 'function_type', 'unknown')
    print(f"Function {i+1}: {result}")
    print(f"           Metadata type: {func_type}")
    print("-" * 60)

Testing random_constant_balanced() function:

------------------------------------------------------------
Function 1: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 2: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 3: CONSTANT (always False) - True: 0, False: 16
           Metadata type: constant
------------------------------------------------------------
Function 4: CONSTANT (always False) - True: 0, False: 16
           Metadata type: constant
------------------------------------------------------------
Function 5: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 6: CONSTANT (always True) - True: 16, False: 0
           Metadata type: constant
------------------------------------------------------------
Function 7: BALANCED - T

### Visualizing a Single Function

Expanding on one function in detail, showing its truth table:

In [9]:
import pandas as pd

def display_truth_table(f: Callable[[bool, bool, bool, bool], bool]) -> pd.DataFrame:
    """
    Create a truth table DataFrame for a 4-input Boolean function.
    
    Args:
        f: A function taking four Boolean arguments
        
    Returns:
        A pandas DataFrame showing all inputs and outputs
    """
    all_inputs = list(product([False, True], repeat=4))
    
    data = []
    for inp in all_inputs:
        data.append({
            'b1': int(inp[0]),
            'b2': int(inp[1]),
            'b3': int(inp[2]),
            'b4': int(inp[3]),
            'f(b1,b2,b3,b4)': int(f(*inp))
        })
    
    return pd.DataFrame(data)


# Generate and display a function's truth table
random.seed(42)  # For reproducibility in demonstration
example_func = random_constant_balanced()

print(f"Function type: {example_func.function_type}")
print(f"\nTruth Table:")
display_truth_table(example_func)

Function type: constant

Truth Table:


,b1,b2,b3,b4,"f(b1,b2,b3,b4)"
0,0,0,0,0,1
1,0,0,0,1,1
2,0,0,1,0,1
3,0,0,1,1,1
4,0,1,0,0,1
5,0,1,0,1,1
6,0,1,1,0,1
7,0,1,1,1,1
8,1,0,0,0,1
9,1,0,0,1,1


---

### The Deutsch Algorithm: A Quantum Approach

The Deutsch algorithm demonstrates quantum advantage for determining if a single-bit Boolean function is constant or balanced. While our `random_constant_balanced` generates 4-bit functions, understanding the simpler 1-bit case (Deutsch's original algorithm) is essential before considering the generalized n-bit Deutsch-Jozsa algorithm.

#### How It Works

For a function $f: \{0,1\} \rightarrow \{0,1\}$, there are exactly 4 possible functions:

| Function | $f(0)$ | $f(1)$ | Type |
|----------|--------|--------|------|
| $f_1$ | 0 | 0 | Constant |
| $f_2$ | 1 | 1 | Constant |
| $f_3$ | 0 | 1 | Balanced |
| $f_4$ | 1 | 0 | Balanced |

**Classical approach**: Requires 2 queries (evaluate $f(0)$ and $f(1)$) to determine if constant or balanced.

**Quantum approach**: Uses superposition and interference to determine the answer with only **1 query**.

#### The Circuit

The Deutsch algorithm circuit:
1. Initialize two qubits: $|0\rangle$ (query) and $|1\rangle$ (ancilla)
2. Apply Hadamard gates to both qubits
3. Apply the oracle $U_f$ (a single query)
4. Apply Hadamard to the first qubit
5. Measure the first qubit: 0 = constant, 1 = balanced

This works due to **phase kickback** - the ancilla qubit in state $|{-}\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ causes a phase shift on the query qubit based on $f(x)$.

### Implementing the Oracle ($U_f$)

The oracle encodes the function $f$ as a quantum gate. For the 4 possible single-bit functions, we implement the oracle as follows:

- **$f_1$** (constant 0): No operation needed (identity)
- **$f_2$** (constant 1): Apply X gate to ancilla
- **$f_3$** (balanced, $f(x) = x$): Apply CNOT with query as control
- **$f_4$** (balanced, $f(x) = \bar{x}$): Apply CNOT + X gate

In [10]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, CXGate


def twobit_function(function_id: int) -> QuantumCircuit:
    """
    Create a quantum oracle circuit for one of the 4 possible single-bit Boolean functions.
    
    The oracle implements U_f: |x⟩|y⟩ → |x⟩|y ⊕ f(x)⟩
    
    Args:
        function_id: Integer 1-4 selecting which function to implement
            1: f(x) = 0 (constant 0)
            2: f(x) = 1 (constant 1)
            3: f(x) = x (balanced, identity)
            4: f(x) = NOT x (balanced, negation)
    
    Returns:
        A QuantumCircuit implementing the oracle
        
    Raises:
        ValueError: If function_id is not in range 1-4
    """
    if function_id not in [1, 2, 3, 4]:
        raise ValueError(f"function_id must be 1, 2, 3, or 4. Got: {function_id}")
    
    # Create a 2-qubit circuit: qubit 0 is query (x), qubit 1 is ancilla (y)
    oracle = QuantumCircuit(2, name=f"U_f{function_id}")
    
    if function_id == 1:
        # f(x) = 0: constant zero
        # Oracle does nothing (y ⊕ 0 = y)
        oracle.id(0)  # Identity gate for clarity
        oracle.id(1)
        
    elif function_id == 2:
        # f(x) = 1: constant one
        # Always flip the ancilla (y ⊕ 1 = NOT y)
        oracle.x(1)
        
    elif function_id == 3:
        # f(x) = x: balanced (identity)
        # Flip ancilla when x=1: CNOT with qubit 0 as control
        oracle.cx(0, 1)
        
    elif function_id == 4:
        # f(x) = NOT x: balanced (negation)
        # Flip ancilla when x=0: X on control, then CNOT, then X to restore
        oracle.x(0)
        oracle.cx(0, 1)
        oracle.x(0)
    
    return oracle


# Display all 4 oracle circuits
print("Oracle circuits for all 4 single-bit Boolean functions:\n")
for i in range(1, 5):
    oracle = twobit_function(i)
    func_type = "Constant" if i <= 2 else "Balanced"
    print(f"Function {i} ({func_type}):")
    print(oracle.draw(output='text'))
    print()

Oracle circuits for all 4 single-bit Boolean functions:

Function 1 (Constant):
     ┌───┐
q_0: ┤ I ├
     ├───┤
q_1: ┤ I ├
     └───┘

Function 2 (Constant):
          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

Function 3 (Balanced):
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

Function 4 (Balanced):
     ┌───┐     ┌───┐
q_0: ┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     



### Deutsch's Algorithm: Complete Implementation

Now we implement the full Deutsch algorithm circuit. The algorithm consists of:

1. **Initialization**: Prepare qubits in state $|01\rangle$
2. **Superposition**: Apply Hadamard gates to create $|{+}{-}\rangle$
3. **Oracle Query**: Apply $U_f$ once (quantum advantage)
4. **Interference**: Apply Hadamard to the first qubit
5. **Measurement**: Measure qubit 0 - result reveals constant (0) or balanced (1)

In [11]:
def deutsch_algorithm(function_id: int) -> QuantumCircuit:
    """
    Implement Deutsch's algorithm for a single-bit Boolean function.
    
    This algorithm determines whether a function f: {0,1} → {0,1} is
    constant or balanced using only ONE query to the oracle.
    
    Args:
        function_id: Integer 1-4 selecting which function to test
            1: f(x) = 0 (constant)
            2: f(x) = 1 (constant)
            3: f(x) = x (balanced)
            4: f(x) = NOT x (balanced)
    
    Returns:
        A QuantumCircuit implementing Deutsch's algorithm
        
    Note:
        Measurement result: 0 = constant function, 1 = balanced function
    """
    # Create circuit with 2 qubits and 1 classical bit for measurement
    qc = QuantumCircuit(2, 1)
    
    # Step 1: Initialize ancilla qubit to |1⟩
    # (query qubit is already |0⟩ by default)
    qc.x(1)
    
    # Step 2: Apply Hadamard gates to both qubits
    # Creates superposition: |+⟩|−⟩
    qc.h(0)  # Query qubit: |0⟩ → |+⟩
    qc.h(1)  # Ancilla qubit: |1⟩ → |−⟩
    
    # Visual barrier to separate initialization from oracle
    qc.barrier()
    
    # Step 3: Apply the oracle (SINGLE QUERY - the quantum advantage!)
    oracle = twobit_function(function_id)
    qc.compose(oracle, inplace=True)
    
    # Visual barrier to separate oracle from measurement prep
    qc.barrier()
    
    # Step 4: Apply Hadamard to query qubit (interference step)
    qc.h(0)
    
    # Step 5: Measure the query qubit
    # Result: 0 = constant, 1 = balanced
    qc.measure(0, 0)
    
    return qc


# Display the complete Deutsch algorithm circuit for function 3 (balanced)
print("Deutsch's Algorithm Circuit (for balanced function f(x) = x):\n")
qc_example = deutsch_algorithm(3)
print(qc_example.draw(output='text'))

Deutsch's Algorithm Circuit (for balanced function f(x) = x):

     ┌───┐      ░       ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■───░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘ ░       ║ 
c: 1/═══════════════════════════╩═
                                0 


### Running the Algorithm: Simulation

We can simulate the quantum circuit using Qiskit's built-in simulator to verify that the algorithm correctly identifies constant vs balanced functions.

In [12]:
from qiskit.primitives import StatevectorSampler


def run_deutsch_algorithm(function_id: int, shots: int = 1024) -> dict:
    """
    Run Deutsch's algorithm and return the measurement results.
    
    Args:
        function_id: Which function to test (1-4)
        shots: Number of measurement shots
        
    Returns:
        Dictionary with measurement counts
    """
    qc = deutsch_algorithm(function_id)
    
    # Use StatevectorSampler for simulation
    sampler = StatevectorSampler()
    job = sampler.run([qc], shots=shots)
    result = job.result()
    
    # Get counts from the result
    counts = result[0].data.c.get_counts()
    return counts


# Test all 4 functions
print("Deutsch's Algorithm Results:\n")
print("=" * 60)

expected_results = {
    1: ("Constant (f=0)", "0"),
    2: ("Constant (f=1)", "0"),
    3: ("Balanced (f=x)", "1"),
    4: ("Balanced (f=NOT x)", "1")
}

for func_id in range(1, 5):
    counts = run_deutsch_algorithm(func_id)
    func_type, expected = expected_results[func_id]
    
    # Determine measured result
    if '0' in counts and '1' in counts:
        measured = "Mixed (should not happen!)"
    elif '0' in counts:
        measured = "0 → Constant"
    else:
        measured = "1 → Balanced"
    
    print(f"Function {func_id}: {func_type}")
    print(f"  Counts: {counts}")
    print(f"  Result: {measured}")
    print(f"  Expected: {expected} ({'Constant' if expected == '0' else 'Balanced'})")
    print("-" * 60)

Deutsch's Algorithm Results:

Function 1: Constant (f=0)
  Counts: {'0': 1024}
  Result: 0 → Constant
  Expected: 0 (Constant)
------------------------------------------------------------
Function 2: Constant (f=1)
  Counts: {'0': 1024}
  Result: 0 → Constant
  Expected: 0 (Constant)
------------------------------------------------------------
Function 3: Balanced (f=x)
  Counts: {'1': 1024}
  Result: 1 → Balanced
  Expected: 1 (Balanced)
------------------------------------------------------------
Function 4: Balanced (f=NOT x)
  Counts: {'1': 1024}
  Result: 1 → Balanced
  Expected: 1 (Balanced)
------------------------------------------------------------


### Summary

In this problem, we implemented:

1. **`random_constant_balanced()`**: A Python function that generates random constant or balanced Boolean functions with 4 inputs. The function returns a callable that can be used as a black-box oracle.

2. **`twobit_function()`**: A Qiskit oracle implementation for the 4 possible single-bit Boolean functions, demonstrating how classical functions are encoded as quantum gates.

3. **`deutsch_algorithm()`**: The complete Deutsch algorithm circuit that determines whether a function is constant or balanced with a **single query** - an exponential speedup over classical algorithms.

**Key Concepts Demonstrated:**
- **Superposition**: Querying all inputs simultaneously using $|{+}\rangle$
- **Phase Kickback**: The ancilla qubit's phase affects the query qubit
- **Interference**: Hadamard gate causes constructive/destructive interference based on function type
- **Quantum Advantage**: Single query vs. classical requirement of 2 queries (or $2^{n-1}+1$ for n-bit functions)